In [2]:
import pandas as pd

In [4]:
items= pd.read_csv('./Data/items.csv')
reviews= pd.read_csv('./Data/Reviews.csv')
games= pd.read_csv('./Data/Steam_games.csv')
users= pd.read_csv('./Data/Users.csv')

In [3]:
print(games['release_year'].max())

2021


In [4]:
games

,app_name,price,id,developer,principal_genre,principal_tag,principal_spec,release_year
0,Lost Summoner Kitty,4.99,761140,Kotoshiro,Action,Strategy,Single-player,2018
1,Ironbound,0.00,643980,Secret Level SRL,Free to Play,Free to Play,Single-player,2018
2,Real Pool 3D - Poolians,0.00,670290,Poolians.com,Casual,Free to Play,Single-player,2017
3,弹炸人2222,0.99,767400,彼岸领域,Action,Action,Single-player,2017
4,Log Challenge,2.99,773570,Sin dato,Sin dato,Action,Single-player,2017
...,...,...,...,...,...,...,...,...
32127,Colony On Mars,1.99,773640,"Nikita ""Ghost_RUS""",Casual,Strategy,Single-player,2018
32128,LOGistICAL: South Africa,4.99,733530,Sacada,Casual,Strategy,Single-player,2018
32129,Russian Roads,1.99,610660,Laush Dmitriy Sergeevich,Indie,Indie,Single-player,2018
32130,EXIT 2 - Directions,4.99,658870,"xropi,stev3ns",Casual,Indie,Single-player,2017


In [5]:
def developer(desarrollador : str ):
    filter_games=games[games['developer']== desarrollador]
    filter_games=filter_games.groupby('release_year').agg(Cantidad_de_Items=('app_name', 'size'),
                                             free_games=('price', lambda x: (x == 0.0).sum())).reset_index()
    filter_games['Contenido Free']= round((filter_games['free_games']/filter_games['Cantidad_de_Items'])*100, 3)
    año= list(filter_games['release_year'])
    cantidad_items= list(filter_games['Cantidad_de_Items'])
    cont_free= list(filter_games['Contenido Free'])
    
    return {'Año':año,'Cantidad de items':cantidad_items, 'Contenido Free':cont_free}

In [6]:
fil= developer('xropi,stev3ns')

In [7]:
fil

{'Año': [2016, 2017],
 'Cantidad de items': [1, 2],
 'Contenido Free': [0.0, 0.0]}

In [8]:
items['users_id'].value_counts()

users_id
js41637              888
maplemage            629
doctr                541
thequeenpanda        524
kube134              476
                    ... 
Bubblecup              1
sithjjf                1
76561198010674657      1
76561198082525427      1
76561198044204914      1
Name: count, Length: 369, dtype: int64

In [9]:
games.head(2)

,app_name,price,id,developer,principal_genre,principal_tag,principal_spec,release_year
0,Lost Summoner Kitty,4.99,761140,Kotoshiro,Action,Strategy,Single-player,2018
1,Ironbound,0.00,643980,Secret Level SRL,Free to Play,Free to Play,Single-player,2018


In [10]:
def userdata( User_id : str ):
    game=games[['id', 'price']]
    user= users[users['user_id']== User_id]
    por_recommed= round((user['recommend_yes']/(user['recommend_yes']+user['recommend_not']))*100,2).iloc[0]
    item_filter= items[items['users_id']==User_id]
    item_filter= item_filter.merge(game, left_on='item_id', right_on='id', how='inner')
    item_filter= item_filter.groupby('users_id').agg({'playtime_forever':'sum',
                                                      'price':'sum'}).reset_index()
    usuario= item_filter['users_id'].iloc[0]
    tiempo= item_filter['playtime_forever'].iloc[0]
    dinero= item_filter['price'].iloc[0]
    
    
    return {'Usuario': usuario, 'Dinero invertido': round(dinero,2), 'Tota horas de juego': tiempo,"% de recomendación":por_recommed}

In [11]:
fil2= userdata('kube134')

In [12]:
fil2

{'Usuario': 'kube134',
 'Dinero invertido': 5451.87,
 'Tota horas de juego': 80061.0,
 '% de recomendación': 100.0}

In [13]:
def UserForGenre(genre: str):
    item = items[['item_id', 'playtime_forever', 'users_id']]
    game = games[['id', 'principal_genre', 'release_year']]
    game = game[game['principal_genre'] == genre]
    filter_game = game.merge(item, left_on='id', right_on='item_id', how='inner')
    user_year= filter_game.groupby(['users_id','release_year']).agg({'playtime_forever': 'sum'}).reset_index()
    max_user= filter_game['users_id'].iloc[0]
    user_playtime_by_year = user_year[user_year['users_id'] == max_user][['release_year', 'playtime_forever']]
    
    year_playtime= user_playtime_by_year.to_dict('records')
    return {"Usuario con más horas jugadas para Género " + genre: max_user, "Horas jugadas": year_playtime}
    

In [14]:
fil3=UserForGenre('Action')


In [15]:
fil3

{'Usuario con más horas jugadas para Género Action': 'UTNerd24',
 'Horas jugadas': [{'release_year': 1983, 'playtime_forever': 23.0},
  {'release_year': 1984, 'playtime_forever': 12.0},
  {'release_year': 1990, 'playtime_forever': 31.0},
  {'release_year': 1993, 'playtime_forever': 43.0},
  {'release_year': 1994, 'playtime_forever': 496.0},
  {'release_year': 1995, 'playtime_forever': 417.0},
  {'release_year': 1996, 'playtime_forever': 35.0},
  {'release_year': 1997, 'playtime_forever': 1230.0},
  {'release_year': 1998, 'playtime_forever': 77.0},
  {'release_year': 1999, 'playtime_forever': 555.0},
  {'release_year': 2000, 'playtime_forever': 18484.0},
  {'release_year': 2001, 'playtime_forever': 503.0},
  {'release_year': 2002, 'playtime_forever': 0.0},
  {'release_year': 2003, 'playtime_forever': 409.0},
  {'release_year': 2004, 'playtime_forever': 22611.0},
  {'release_year': 2005, 'playtime_forever': 3695.0},
  {'release_year': 2006, 'playtime_forever': 261.0},
  {'release_year': 

In [16]:
def best_developer_year( año : int ):
    game=games[['developer','app_name','id','principal_genre','release_year']]
    game=game[game['release_year']== año]
    review=reviews[['item_id','senti_positivo']]
    reviews_filter= game.merge(review, left_on='id', right_on='item_id', how= 'inner') 
    reviews_filter= reviews_filter[['developer','app_name','senti_positivo']]
    reviews_filter=reviews_filter.groupby(['developer','app_name']).agg({'senti_positivo':'sum'}).reset_index()
    reviews_filter= reviews_filter.sort_values(by= 'senti_positivo', ascending= False)
    
    developer= list(reviews_filter.iloc[:,0])
    app= list(reviews_filter.iloc[:,1])
    positive=list(reviews_filter.iloc[:,2])
    
    
    return [{'Puesto 1': [{'Developer':developer[0]},{'App':app[0]},{'Reviews positive':positive[0]}]},{'Puesto 2': [{'Developer':developer[1]},{'App':app[1]},{'Reviews positive':positive[1]}]},
            {'Puesto 3': [{'Developer':developer[2]},{'App':app[2]},{'Reviews positive':positive[2]}]}]

In [17]:
fil4= best_developer_year(2002)

In [18]:
fil4

[{'Puesto 1': [{'Developer': 'Bethesda Game Studios'},
   {'App': 'The Elder Scrolls III: Morrowind® Game of the Year Edition'},
   {'Reviews positive': 14}]},
 {'Puesto 2': [{'Developer': 'Rockstar Games'},
   {'App': 'Grand Theft Auto III'},
   {'Reviews positive': 6}]},
 {'Puesto 3': [{'Developer': 'FireFly Studios'},
   {'App': 'Stronghold Crusader HD'},
   {'Reviews positive': 3}]}]

In [19]:
def developer_reviews_analysis( desarrolladora : str ):
    game= games[['id','developer']]
    game= game[game['developer']== desarrolladora]
    review= reviews[['item_id', 'senti_negativo','senti_positivo']]
    game_filter= game.merge(review, left_on='id', right_on='item_id', how='inner')
    game_filter= game_filter.groupby('developer').agg({'senti_negativo':'sum',
                                                       'senti_positivo':'sum'}).reset_index()
    developer= game_filter['developer'].iloc[0]
    review_neg= game_filter['senti_negativo'].iloc[0]
    review_pos= game_filter['senti_positivo'].iloc[0]
    
    return {developer:[{'Negativas':review_neg}, {'Positivas': review_pos}]}
        
    

In [20]:
fil5= developer_reviews_analysis('Bethesda Game Studios')

In [21]:
fil5

{'Bethesda Game Studios': [{'Negativas': 77}, {'Positivas': 228}]}

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [153]:
def recomendacion_juego(id_de_producto: int):
    atributos = games[games['id'] == id_de_producto]
    
    if atributos.empty:
        return 'No se encontró un juego con el ID proporcionado, prueba otro ID.'
    
    juegos = games[games['principal_genre'] == atributos['principal_genre'].iloc[0]]
    juegos= juegos[['app_name','principal_genre','principal_tag','principal_spec']]
    juegos['caracteristicas'] = juegos[['principal_genre', 'principal_tag', 'principal_spec']].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
    juegos= juegos[['app_name','caracteristicas']].iloc[:1001,:]
    vector= TfidfVectorizer()
    matriz= vector.fit_transform(juegos['caracteristicas'])
    matriz_similaridad= cosine_similarity(matriz)
    indice_juego = juegos.index[juegos['app_name'] == atributos['app_name'].iloc[0]].tolist()[0]
    similares = list(enumerate(matriz_similaridad[indice_juego]))
    similaridades = sorted(similares, key=lambda x: x[1], reverse=True)[1:6]
    juegos_recomendados = [juegos.iloc[i[0]]['app_name'] for i in similaridades]
    
    return {'Juegos similares que te pueden interesar': juegos_recomendados}
    
    

In [154]:
fil6= recomendacion_juego(413120)

In [155]:
fil6

{'Juegos similares que te pueden interesar': ['Battle Royale Trainer',
  'Army of Tentacles: (Not) A Cthulhu Dating Sim: Black GOAT of the Woods Edition',
  'The Warrior Of Treasures',
  'Fallen Mage',
  'Twisted Enhanced Edition']}

In [48]:
#atributos = games[games['id'] == 768800]
#atributos

,app_name,price,id,developer,principal_genre,principal_tag,principal_spec,release_year,carteristicas
9,Race,8.866131,768800,RewindApp,Casual,Indie,Single-player,2018,Casual-Indie-Single-player


In [50]:
#juegos = games[games['principal_genre'] == atributos['principal_genre'].iloc[0]]
#juegos= juegos[['app_name','principal_genre','principal_tag','principal_spec']]
#juegos['carteristicas'] = juegos[['principal_genre', 'principal_tag', 'principal_spec']].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
#juegos

,app_name,principal_genre,principal_tag,principal_spec,carteristicas
2,Real Pool 3D - Poolians,Casual,Free to Play,Single-player,Casual-Free to Play-Single-player
9,Race,Casual,Indie,Single-player,Casual-Indie-Single-player
13,Beach Rules,Casual,Casual,Single-player,Casual-Casual-Single-player
14,Planetarium 2 - Zen Odyssey,Casual,Indie,Single-player,Casual-Indie-Single-player
31,Lost Cities,Casual,Casual,Single-player,Casual-Casual-Single-player
...,...,...,...,...,...
32123,Raining blocks,Casual,Indie,Single-player,Casual-Indie-Single-player
32125,BAE 2,Casual,Indie,Single-player,Casual-Indie-Single-player
32127,Colony On Mars,Casual,Strategy,Single-player,Casual-Strategy-Single-player
32128,LOGistICAL: South Africa,Casual,Strategy,Single-player,Casual-Strategy-Single-player


In [54]:
#vector= TfidfVectorizer()
#matriz= vector.fit_transform(juegos['carteristicas'])
#matriz_similaridad= cosine_similarity(matriz)
#print(matriz_similaridad)

[[1.         0.14080567 0.23006966 ... 0.11541    0.11541    0.14080567]
 [0.14080567 1.         0.54403672 ... 0.2729055  0.2729055  1.        ]
 [0.23006966 0.54403672 1.         ... 0.44591441 0.44591441 0.54403672]
 ...
 [0.11541    0.2729055  0.44591441 ... 1.         1.         0.2729055 ]
 [0.11541    0.2729055  0.44591441 ... 1.         1.         0.2729055 ]
 [0.14080567 1.         0.54403672 ... 0.2729055  0.2729055  1.        ]]


In [56]:
#indice_juego = juegos.index[juegos['app_name'] == atributos['app_name'].iloc[0]].tolist()[0]
#indice_juego

9

In [57]:
#similares = list(enumerate(matriz_similaridad[indice_juego]))
#similares

[(0, 0.1408056679925726),
 (1, 1.0000000000000002),
 (2, 0.5440367173761865),
 (3, 1.0000000000000002),
 (4, 0.5440367173761865),
 (5, 0.5440367173761865),
 (6, 0.14968714067163097),
 (7, 0.5440367173761865),
 (8, 0.5440367173761865),
 (9, 1.0000000000000002),
 (10, 0.5440367173761865),
 (11, 0.15526363818469036),
 (12, 1.0000000000000002),
 (13, 0.5440367173761865),
 (14, 0.5440367173761865),
 (15, 0.5440367173761865),
 (16, 0.5440367173761865),
 (17, 0.5440367173761865),
 (18, 0.5440367173761865),
 (19, 0.5440367173761865),
 (20, 0.5440367173761865),
 (21, 0.5440367173761865),
 (22, 0.5440367173761865),
 (23, 0.5440367173761865),
 (24, 0.5440367173761865),
 (25, 0.5440367173761865),
 (26, 0.27290550224944277),
 (27, 0.15526363818469036),
 (28, 0.5440367173761865),
 (29, 0.19284405884762668),
 (30, 0.5440367173761865),
 (31, 0.21332163438040896),
 (32, 0.5440367173761865),
 (33, 0.5440367173761865),
 (34, 0.5440367173761865),
 (35, 0.5440367173761865),
 (36, 0.10562895115664378),
 (37

In [58]:
#similaridades = sorted(similares, key=lambda x: x[1], reverse=True)[1:6]
#similaridades

[(3, 1.0000000000000002),
 (9, 1.0000000000000002),
 (12, 1.0000000000000002),
 (48, 1.0000000000000002),
 (69, 1.0000000000000002)]

In [59]:
#juegos_recomendados = [juegos.iloc[i[0]]['app_name'] for i in similaridades]
#juegos_recomendados

['Planetarium 2 - Zen Odyssey',
 'RIP - Trilogy™',
 'Ninja Reflex: Steamworks Edition',
 'Fluttabyes',
 'Arcadia']

In [240]:
def recomendacion_usuario( id_de_usuario: str):
    items[['item_id','users_id']]
    item= items[items['users_id']==id_de_usuario]
    ids= item['item_id']
    item= item.merge(games, left_on='item_id', right_on='id', how= 'inner')
    item= item[['principal_genre','item_id']]
    genre= pd.DataFrame(item['principal_genre'].value_counts())[:2]
    genre= genre.index.tolist()
    
    game= games[games['principal_genre'].isin(genre)][['app_name','id','principal_genre','principal_tag','principal_spec']]
    game= game[~game['id'].isin(ids.tolist())].iloc[:1001,:]
    game['caracteristicas']= game[['principal_genre', 'principal_tag', 'principal_spec']].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
    game=game[['app_name','id','caracteristicas']]
    
    vector= TfidfVectorizer()
    matriz= vector.fit_transform(game['caracteristicas'])
    matriz_similaridad= cosine_similarity(matriz)
    similares = list(enumerate(matriz_similaridad))
    similaridades = sorted(similares, key=lambda x: x[0], reverse=True)[1:6]
    juegos_recomendados = [game.iloc[i[0]]['app_name'] for i in similaridades]
    
    return {'Juegos similares que te pueden interesar': juegos_recomendados}

In [242]:
fil7= recomendacion_usuario('thequeenpanda')
fil7

{'Juegos similares que te pueden interesar': ['White Noise Online',
  'Retro Game Crunch',
  'Paradigm Shift',
  'Sweet Lily Dreams',
  'Toribash']}

In [197]:
user=items['users_id'].value_counts()
user

users_id
js41637              888
maplemage            629
doctr                541
thequeenpanda        524
kube134              476
                    ... 
76561198074200011      4
76561198089077856      4
76561198157999555      3
76561198215949741      2
jimmyarcexd            1
Name: count, Length: 269, dtype: int64